In [1]:
# Dependencies
import os
import pandas
import math
import time
import nltk
nltk.download('punkt')

import torch
from torch.utils.data import DataLoader, TensorDataset
device = torch.device('cpu')

from transformers import pipeline
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW
from huggingface_hub import hf_hub_download
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rudy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# summarizer = pipeline("summarization", model="sambydlo/scientific_abstract_simplification-tomasg25/scientific_lay_summarisation")
model_name = [
                "sambydlo/bart-large-scientific-lay-summarisation",
                "com3dian/Bart-large-paper2slides-summarizer",
                "philschmid/bart-large-cnn-samsum"
            ]

In [3]:
tokenizer = BartTokenizer.from_pretrained(model_name[0])
model = BartForConditionalGeneration.from_pretrained(model_name[0]).to(device)

In [4]:
# Dezen & Flores '18
reference = """We present numerical spectral and vertical structure calculations appropriate for 
            near-Eddington luminosity, radiation pressure dominated accretion disks
            around stellar mass black holes. We cover a wide range of black hole spins, and
            incorporate dissipation profiles based on first-principles three-dimensional MHD
            disk interior simulations. We also include non-zero stresses at the ISCO, which
            results in the disk effective temperature to increase rapidly towards the black
            hole, and give rise to rather extreme conditions with high temperatures and low
            surface densities. We found that local annuli spectra become increasingly characteristic 
            of saturated Comptonisation with decreasing distance to the black hole.
            While the spectra becomes harder with increasing black hole spin, they do not
            give rise to a broad power law tail even at maximum spin. We discuss the implications 
            of our results in the context of the steep power law (SPL) state and the
            associated high-frequency quasi-periodic oscillations (HFQPO) observed in some
            X-ray binary systems"""
introduction = """Galactic black hole X-ray binaries (BHB) show several states of outburst distinguished
by luminosity, spectral shape and variability (see for example, McClintock & Remillard
(2006) and Done, Gierlinski, & Kubota (2007)). In particular, at their highest luminosities
the spectra contains a steep power law component with photon index Γ > 2.4 (McClintock & Remillard
2006). These energetically significant power law tails begin at the spectral peak (≈ 10 keV)
and could extend into the MeV regime (Ling & Wheaton 2005; Grove et al. 1998). Moreover, this steep power 
law (SPL) spectral state is accompanied by high-frequency (ν > 50 Hz)
quasi-periodic oscillations (HFQPO) in the light curves when integrated over approximately
10 to 30 keV in photon energies.
Understanding the first-principles physics of radiating accretion flows that presumably
underly these observational properties remain an important outstanding problem in astrophysics. The standard thin accretion disk model (Shakura & Sunyaev 1973; Novikov & Thorne
1973; Riffert & Herold 1995) assumed that the stress and luminosity at the innermost stable
circular orbit (ISCO) drops to zero, and that at this point the material essentially simply disappears into the black hole. This assumption received significant recent theoretical scrutiny
upon the realization that magnetohydrodynamic turbulence (Balbus and Hawley 1991, 1998)
is probably the source of stress that drives accretion. In particular, Agol and Krolik (2000)
demonstrated that having non-zero magnetic stresses at the ISCO can cause the effective
temperature to rise sharply towards the black hole instead of fall to zero as predicted by
the standard model. Among the potentially observable consequences postulated by these
authors, the inner disk consequently becomes effectively thin, and extends the spectrum to
higher frequencies.
More recently, Dexter & Blaes (2014) (from here on referred to as DB14) proposed that
the Agol and Krolik (2000) model provides feasible mechanism for explaining both the steep
power law (SPL) state seen at near Eddington luminosities and the associated high frequency
quasi-periodic oscillations (HFQPO). These authors argued that the rapidly rising effective
temperature with decreasing distance to the black hole would give rise to the SPL spectra,
while also providing a natural filter for the HFQPOs that do not require the entire disk to
oscillate coherently.
In this work, we undertake a detailed numerical study of the structure and spectra of
near-Eddington accretion disks with non-zero magnetic stresses the ISCO, and particularly
focus on the effects of black hole spin. Unlike previous efforts that relied on one-zone models,
we self-consistently couple vertical structure to radiative transfer at each disk annuli, and
generate spectra that fully incorporates effects Comptonisation and metal opacities. Our
inputs are time and horizontally averaged vertical dissipation profiles from first-principles
stratified shearing-box simulations of accretion flows (Hirose, Krolik & Blaes 2009). These
calculations evolve the time-dependent three-dimensional radiation magneto-hydrodynamic
equations and accounts for the tidal vertical gravity from the black hole. In simulations over
a wide range of box-integrated radiation to gas pressure ratios, the resulting vertical spatial
dissipation profiles generally peak at around a pressure scale-height away from the disk
mid-plane, and should capture the effects of MRI turbulence. Moreover, these simulations
collectively indicate that the α-prescription (Shakura & Sunyaev 1973) relationship between
pressure and stress approximately hold (Hirose, Blaes & Krolik 2009). This means we are 
justified, at least in light of recent simulations, to use the α-model with modifications to
account for non-zero inner torque to generate radial profiles of total surface density Σ0 and
effective temperature Teff that are also necessary for our vertical structure and radiative
transfer computations.
This paper is organized as follows. In section 2 we outline our numerical methods, paying
particular attention to how we incorporated non-zero inner torque. Section 3 showcases our
numerical results, including full-disk spectra for all black hole spin values we covered. We
turn to the possibility of HFQPOs in section 4, and conclude in section 5 with a discussion
of on-going and future work.
"""
methods = """
We compute one-dimensional, time-independent and non-LTE disk annuli vertical structure and spectral models using the stellar atmosphere code TLUSTY (Hubeny & Lanz 1995)
that has been adapted for accretion disk applications (Hubeny & Hubeny 1998; Hubeny et al.
2000). The program self-consistently solves the disk vertical structure equations along with
angle-dependent and multi-frequency radiative transfer, and does not assume that the spectrum is modified from a blackbody via spectral hardening factors. We treat electron (both
Thomson and Compton) scattering with an angle-averaged Kompaneets source function
(Hubeny et al. 2001), and allow for deviations from local thermodynamic equilibrium by
explicitly computing ion populations for hydrogen, helium and metals.
TLUSTY self-consistently calculates the vertical structure of and local emergent spectrum from an accretion disk annuli. To specify an annuli, TLUSTY requires as input the
mid-plane surface density Σ0 (Figure 1), effective temperature Teff (Figure 2), and orbital
frequency squared Ω2
, all of which are functions of distance from the black hole. Interested readers can consult previous TLUSTY-based work such as Davis et al. (2005) and
Tao & Blaes (2013) for more details on using and modifying the code, as well as further
discussions of numerical considerations. Next, we focus on how we generated these input
parameters for a given blackhole mass and spin while incorporating both non-zero torque at
disk inner boundary and simulation based dissipation physics.
We made several assumptions and approximations may impact the disk spectra and
vertical structure that we will compute. First, we characterized the ionized plasma with a
single gas temperature Tg because the mean free path of electron-ion collisions is much less
than disk scale height, and therefore the electrons and protons are thermally locked. Further,
we only consider vertical radiative diffusion for energy transport, so that we can relate the
dissipation rate Q to the frequency-integrated flux F via
Q(z) = dF
dz , (1)
where z is the height measured vertically upward (with z = 0 at mid-plane). By analyzing
several shearing box simulations, Blaes et al. (2011) found that radiative diffusion is probably
not the only important vertical energy transport mechanism. Specifically, radiation pressure
work and magnetic buoyancy driven radiation advection can be as important as diffusive
flux near the mid-planes of radiation pressure dominated disks. However, these simulations
also showed that radiative diffusion still dominates at higher altitudes, so that in the end it
is still the primary means to carry dissipated energy into the photospheric region where the
spectrum forms. We therefore suspect that the emergent spectra will not change drastically
even if we included radiative advection. On the other hand, more recent global simulations
found radiative advection to be important even in the outflow regions of super-Eddington
accretion flows (Jiang, Stone & Davis 2014), but how these results would impact the spectra
is not yet clear. Finally, we ignore potentially important the effects of irradiation and outflow.
We also do not include magnetic fields in this study. While simulations such as those described in Hirose, Krolik & Blaes (2009) found that magnetic pressure can be just as important, and even dominate over, radiation pressure in disk outer layers. However, Davis et al.
(2009) showed that the expected hardening of the spectra due to significant magnetic support against gravity is largely compensated by the softening effects of complex density inhomogeneities around the photospheric regions. Moreover, radiative transfer calculations
incorporating simulation-based vertical magnetic acceleration (Tao & Blaes 2013) showed
that dissipation profiles is what primarily drove significant changes in spectral shapes.
To generate input for TLUSTY and study the effects of non-zero inner torque, we
use the results of Agol and Krolik (2000) as implemented by DB14 to numerically compute
the vertically integrated surface density Σ0 (Figure 1), effective temperature Tef f (Figure
2) and orbital frequency Ø as a function of distance from the black hole by solving onezone relativistic radial structure equations. Compared to the standard thin disk models
(Novikov & Thorne 1973), Agol and Krolik (2000) adds a single parameter ∆ǫ that characterizes the additional radiative efficiency (relative to one computed in terms of only the
binding energy) due to including a non-zero torque at the disk’s inner edge. Our modeled
disks accrete onto a 7 solar mass black hole with α = 0.02 (Shakura & Sunyaev 1973). We
chose the normalized accretion rates
m˙ =
ǫM c ˙ 2
LEdd
=
(ǫ0 + ∆ǫ)M c ˙ 2
LEdd
(2)
such that the disk luminosity is approximately Eddington, where ǫ0 is the accretion efficiency
without enhancement due to the inner torque. Our models cover several black hole spins
ranging from a/M = 0.4 to a/M = 0.99. For all spin values, we set ∆ǫ to 0.1. We divide
each disk into many (on the order of 50, and vary slightly between models) annuli starting
at the ISCO. Finally, we use a relativistic transfer code (Agol 1997) to obtain the full disk
spectra as seen by distant observers.
Fig. 1.— Total disk surface density Σ0 as a function of distance from black hole for a/M =
0.4. Note that surface density continues to decrease towards the ISCO when ∆ǫ = 0.1 (black)
but rises sharply for the disk with ∆ǫ = 0 (red).
We incorporate simulation-based dissipation physics by writing the vertical dissipation
profile as a function of fractional surface density. Specifically, we fit the horizontal and time
averaged dissipation profile from Hirose, Krolik & Blaes (2009) using the broken power-law
expression (Tao & Blaes 2013)
QΣ
ρ
= −Σ
dF
dΣ
= F0



A

Σ
Σ0
0.5
, Σ/Σ0 < 0.11
B

Σ
Σ0
0.2
, Σ/Σ0 > 0.11
. (3)
as shown in Figure 3, where the power-law indices and the Σ/Σ0 = 0.11 division point are
all required input for TLUSTY. Here F0 (obtained from Teff) and Σ0 are the total emergent
flux and mid-plane surface density, respectively. The code automatically computed the
normalization constants A(≈ 0.65) and B(≈ 0.33) to ensure that the profile is continuous at
Fig. 2.— Representative radial effective temperature profiles as a function of distance from
the black hole normalized to the gravitational radii for accretion disks with zero (solid) and
non-zero inner torque (dotted), respectively. The colors correspond to models with different
black hole spins: blue, a/M = 0.4; green, a/M = 0.6; black, a/M = 0.7; red, a/M = 0.99.
Note that the effective temperature continues to rise sharply towards the black hole all the
way down to the inner edge for the black curve.
the division point, and that the total flux is F0 so that
F0 =
Z Σ0
0
dF
dΣ
dΣ. (4)
The surface density Σ at height z is given by
Σ(z) = Z ∞
z
ρ(z
′
)dz′
, (5)
where ρ is the depth dependent density. We define z = 0 to be the disk mid-plane, and
hence ρ and Σ both go to zero at infinity. In a previous TLUSTY calculation, Davis et al.
(2005) used a qualitatively similar prescription with a power law index of 0.5 in the entire
vertical disk domain. Here we fit the simulation in more detail, resulting in the break at
Σ/Σ0 = 0.11. This dissipation profile presumably captures the effects of turbulence due to
the magnetorotational instability (Balbus and Hawley 1991).
Before moving on, we note that a slim (Abramowicz et al. 1988), rather than thin, disk
model is likely more appropriate for high luminosity systems radiating at near or beyond the
Fig. 3.— Dissipation per unit mass normalized to total flux as a function of fractional
surface density for a representative accretion disk annulus. The blue curve denotes the
broken-power law expression of equation (3), while the red curve is the horizontal and time
averaged dissipation profile from simulation 1112a of Hirose, Krolik & Blaes (2009). While
equation (3) gives QΣ/ρ, here we instead graphed Q/ρ to emphasize the dissipation per unit
mass in the upper layers.
Eddington limit. Moreover, we ignored the likely important effects of radial advection associated with such slim disks, and computed the radial profile essentially from height-averaged
quantities. Properly incorporating these effects requires two-dimensional slim disk models
that couple the radial and vertical structures, such as those proposed in Sadowski et al.
(2011). At present, these models do not yet include self-consistent multi-frequency radiative
transfer.
"""


In [5]:
# arXiv:quant-ph/0302169v1  21 Feb 2003PROCEEDINGS OF THE TENTH INTERNATIONAL CONFERENCE ON
# COMPUTATIONAL AND APPLIED MATHEMATICS
test_doc = """
1.Introduction. Modern physics returns to some of its origins dating back
to the ﬁrst part of the last century. Examples are entanglement, according to
Schr¨ odinger themost peculiar property occuring inquantum mech anics, orthe
condensationofvery coldatomspredictedbyEinsteinandBose(Bo se-Einstein
condensate). Another example is nonlinear dynamics and chaos, da ting back
to the work of Poincar´ e and others and its modern descendents - quantum
chaos. Entanglement plays a fundamental role in quantum optics, q uantum
computing, quantum cryptography, and quantum teletransport ation [1, 2].
TheBose-Einsteincondensateisanewstateofmatter. Classicalc haosplaysan
important role in macroscopic physics, chemistry and biology. Quant um chaos
may become quite important for example in semiconductor devices, w hen due
to decreasing size (from micron to Angstrom) atomic length scales a re reached
and the laws of quantum mechanics apply. Like chaotic motion in a ﬂuid,
quantum chaos may cause a similar phenomenon for the ﬂux of the ele ctric
current in a semiconductor.
What has the quantum action - the subject of this article - to do with the
aboveissues? Thequantumactionisanewconceptconcerningthef oundations
of quantum mechanics. Since de Broglie we know that the quantum wo rld has
apparently a dual face - a particle picture and a wave picture. The q uantum
action can be considered as a link between classical physics and quan tum
physics. This may be interpreted as if the quantum world has a third f ace -
a classical picture with a new interaction. Consequently, the quant um action
may be expected to be useful to better understand the relation b etween and
the transition from classical physics to quantum physics. In partic ular, there
are some very fruitful concepts in quantum physics, which are der ived from
1991Mathematics Subject Classiﬁcation. Primary: 81Q50, 37F25.
Key words and phrases. Nonlinear dynamics, quantum chaos.
1
2 H. KR¨OGER
classical physics: One example is the instanton solution in a quantum ﬁ eld
theory. Another example is quantum chaos.
On the other hand there are phenomena occuring in the quantum wo rld
which do not have a counter part in the classical world. A well known e xample
is tunneling: A double well potential with a barrier in the middle prohibits in
classical physics a transition from one minimum to the other if the ene rgy is
too small to overcome the barrier. However, in quantum physics a t ransition
does occur with a certain probability. It is noteworthy that tunnelin g and
instantons are closely related [3]. There are other systems, where transitions
are classically forbidden, due to some conserved quantity (other t han energy),
but quantum mechanical transitions are possible. This is known as dy namical
tunneling [4]. It has recently been realized experimentally [5, 6]. It tur ns
out that quantum chaos seems to facilitate dynamical tunneling. We suggest
that the quantum action [7, 8, 9, 10, 11] is an appropriate tool to in vestigate
phenomena on the borderline between classical and quantum physic s and to
analyze quantum tunneling, quantum instantons and quantum chao s.
In thefollowing we discuss theconcept of thequantum action. Inpa rticular,
we discuss analytical properties of the quantum action in the limit whe n the
time of a quantum mechanical transition goes to inﬁnity. As an examp le of the
use of the quantum action we discuss quantum instantons from a do uble well
potential and quantum chaos from the 2-D anharmonic oscillator. F inally, we
give an outlook on further use of the quantum action.
2.The problem of quantum chaos and quantum instantons. Chaos
is quite well understood in classical physics. Its origin in terms of non linear
dynamics has been investigated thoroughly in theoretical physics a nd math-
ematics. If one considers nature at the level of atoms, where the laws of
quantum mechanics (Q.M.) hold, chaotic phenomena are by far less we ll un-
derstood than in classical (macroscopic) physics [12, 13, 14, 15, 1 6]. Quantum
chaos is usually analyzed in terms of the level density of its energy sp ectrum.
This is based on a conjecure by Bohigas et al. [17] which states that t he level
density distribution (Poissonian vs. Wignerian) corresponds to an in tegrable
(non-chaotic) vs. chaotic quantum system. In classically chaotic s ystems,
phase space portraits, Poincar´ e sectioons, Lyapunov exponen ts and the KAM
theorem are very useful to analyze the chaotic behavior. Howeve r, those no-
tions have no direct counterpart in quantum systems. The problem is due to
thefact that inquantum mechanics one cannot specify with certain ty position
and momentum at the same time (Heisenberg’s uncertainty relation) . Hence
a sharp point in phase space has no meaning.
This is also the problem with the deﬁnition of quantum instantons. Rec all
that in classical physics the instanton is a solution in imaginary time whe re
a particle moves in a potential with degenerate minima, such that it st arts
at one minimum (sharp position and velocity zero) and goes over to an other
3
minimum (sharp position and velocity zero). Again boundary condition s are
sharp points in phase space with no direct analogon in Q.M.
Here we take the following approach: Via the quantum action one can pa-
rametrize quantum mechanical transition amplitudes (which contain all infor-
mation about the physics of the system). On the other hand, the q uantum
action has the same mathematical form as the classical action, a kin etic term
(proportional to velocity squared) and a local potential term. Ho wever, the
action parameters, like mass and potential are diﬀerent from the c lassical one,
in general. Starting from such quantum action then allows to constr uct a
phase space portrait and hence reintroduce the tools of classical chaos theory
to the quantum world.
Let us come back to the problem of a point in phase space used as initia l
condition. Recall in classical mechanics that equations of motions ar e ﬁxed
by specifying the dynamics (usually in terms of a diﬀerential equation ) and
by specifying initial conditions. Usually one speciﬁes position and veloc ity at
some initial time. A consequence of this type of boundary conditions is that
certain trajectories are forbidden. For example those where the energy is too
small to overcome a potential barrier. Moreover, this initial condit ion has no
direct analogon in Q.M. However, there is a simple way out: Consider cla ssical
mechanics with 2-point boundary conditions. I.e., one speciﬁes posit ions at
some initial and ﬁnal time. Then there are no classically forbidden tra jecto-
ries (sometimes even inﬁnitely many trajectories are possible). In q uantum
physics, there is a well deﬁned transition amplitude. Below we deﬁne t he
quantum action by use of two-point boundary conditions.
3.Postulate of quantum action. The quantum action [7] is deﬁned by the
following requirements. For a given classical action
S[x] =/integraldisplay
dtm
2˙x2−V(x), (1)
there is a quantum action
˜S[x] =/integraldisplay
dt˜m
2˙x2−˜V(x), (2)
which parametrizes the Q.M. transition amplitude
G(xf,tf;xi,ti) =˜Zexp/bracketleftbiggi
/planckover2pi1˜Σ/vextendsingle/vextendsingle/vextendsinglexf,tf
xi,ti/bracketrightbigg
,
˜Σ/vextendsingle/vextendsingle/vextendsinglexf,tf
xi,ti=˜S[˜xcl]/vextendsingle/vextendsingle/vextendsinglexf,tf
xi,ti=/integraldisplaytf
tidt˜m
2˙˜x2
cl−˜V(˜xcl)/vextendsingle/vextendsingle/vextendsingle/vextendsinglexf
xi.
4 H. KR¨OGER
where ˜xcldenotes the classical path corresponding to the action ˜S. We require
2-point boundary conditions,
˜xcl(t=ti) =xi
˜xcl(t=tf) =xf.
˜Zstands for a dimensionful normalisation factor. Note: The parame ters of
the quantum action (mass, potential) are independent of the boun dary points
xf,xi, but depend on the transition time T=tf−ti. The same q-action
parametrizes all transition amplitudes for a given transition time.
The postulateleads tothequestion: Doessuchquantum actionexis t? Sofar
we have no general proof. However, the answer is aﬃrmative in the following
cases:
(a) Harmonic oscillator. In this case the classical action satisﬁes th e deﬁnition
of the quantum action [18, 7], hence both coincide.
(b) Arbitrary local potential, limit when transition time T→0: Then the
quantum action exists as it coincides with classical action.
(c) Arbitrary potential, imaginary time, limit when transition time T→ ∞
(equivalent to temperature going to zero): Then the quantum act ions exists,
beingdiﬀerentfromtheclassical action, ingeneral(aproofisgiven inRef.[11]).
4.Analytical properties of the quantum action. In this limit of large
imaginary time, the quantum action has a number of remarkable prop erties:
(i)TheWKBapproximationforthegroundstatewavefunctionbeco mesexact,
after replacing the classical action by the quantum action.
(ii) There is an analytic expression for the ground state wave funct ion in terms
of the quantum action.
(iii) The ground state energy coincides with the minimum of the quantu m
potential.
(iv) The ground state wave function has a maximum exactly at the sa me
position, where the quantum potential has a minimum.
(v) The quantum action allows also to reproduce energies and wave f unctions
of excited states. Example: The spectrum of the hydrogen atom, considering
the lowest energy states for given angular momentum.
Let us detail some of those results.
For physical and mathematical reasons, it is interesting to go from real time
over to imaganiary time. A mathematical reason is that the path inte gral for
the Q.M. transition amplitude then becomes well deﬁned (Wiener integ ral).
A physical reason is that the description of ﬁnite temperature phy sics requires
the use of imaginary time. Thus we make the following transition
t→ −it .
5
Then the action becomes the so-called Euclidean action
SE=/integraldisplayT
0dtm
2˙x2+V(x), (3)
the transition amplitude becomes the Euclidean transition amplitude
GE(xf,T;xi,0) =/angbracketleftxf|exp[−HT//planckover2pi1]|xi/angbracketright
=/integraldisplay
[dx]exp[−SE[x]//planckover2pi1]/vextendsingle/vextendsingle/vextendsingle/vextendsinglexf,T
xi,0,
and the quantum action becomes the Euclidean quantum action
GE(xf,T;xi,0) =˜ZEexp/bracketleftbigg
−1
/planckover2pi1˜ΣE/vextendsingle/vextendsingle/vextendsinglexf,T
xi,0/bracketrightbigg
. (4)
Let us recall the Feynman-Kac formula, which makes a statement a bout the
time evolution of the quantum system in the limit of large imaginary time.
In this limit the time evolution of a quantum system is determined by the
behavior of exp[ −HT//planckover2pi1] whenTgoes to∞. In particular, one has
e−HT//planckover2pi1−→T→∞|ψgr/angbracketrighte−EgrT//planckover2pi1/angbracketleftψgr|,
whereEgrandψgrarethe groundstate energy andwave function, respectively.
Now let us consider the case of a potential V(x) having a unique minimum
and obeying V(x)→ ∞whenx→ ±∞. We consider the Euclidean action
in the limit of large time. The trajectory minimizes the action. Because the
kinitic term and the potential term (by assumption) are positive, th is means
that the trajectory minimizes ﬂuctuations and stays as close as po ssible to the
bottom of the potential valley. This implies for T→ ∞,
˜V→˜Vmin,˜Tkin→0, ǫ=−˜Tkin+˜V→˜Vmin,
and
˜Σ≡˜S[˜xcl]|xf,T
xi,0=/integraldisplayT
0dt˜Tkin+˜V= ˜v0T+/parenleftbigg/integraldisplay0
xi+/integraldisplayxf
0dx±/radicalBig
2˜m(˜V(x)−˜Vmin)/parenrightbigg
,
where the sign depends on initial and ﬁnal data. The transition amplit ude
then becomes
G(xf,T;xi,0) =˜Zexp[−˜Σ//planckover2pi1]/vextendsingle/vextendsingle/vextendsinglexf,T
xi,0T→∞−→˜Z0exp[−˜v0T//planckover2pi1]
×exp[−/integraldisplayxfi
0dx/radicalBig
2˜m(˜V(x)−˜Vmin)//planckover2pi1]exp[−/integraldisplay0
xindx/radicalBig
2˜m(˜V(x)−˜Vmin)//planckover2pi1].
By comparison with the Feynman-Kac formula one obtains the followin g ana-
lytic expressions for the ground state energy and wave function, expressed in
terms of the quantum action,
Egr=˜Vmin, ψgr(x) =1
Ne−/integraltext|x|
0dx′√
2˜m(˜V(x′)−˜Vmin)//planckover2pi1. (5)
6 H. KR¨OGER
Combining this with the Schr¨ odinger equation leads to the following tr ansfor-
mation law
2m(V(x)−Egr) = 2˜m(˜V(x)−˜Vmin)−/planckover2pi1
2d
dx2˜m(˜V(x)−˜Vmin)/radicalBig
2˜m(˜V(x)−˜Vmin)sgn(x).(6)
Although those results have been obtained in imaginary time, they ho ld also
in real time (ground state energy Egrand wave function ψgr(x) are the same
as in real time).
Eq.(5) shows that the minimum of the quantum potential coincides wit h
the ground state energy. Second, the position of the minimum of th e quantum
potential coincides with the position of the maximum of the ground st ate
wave function. This is shown in Fig.[1]. Hence the quantum potential giv es a
much better picture of the behavior of the quantum system than t he classical
potential, which does not share those properties.
0 2024
Egr Particle□in□the□potential□:□V(x)□=□m ω2x2/2+g/x 2Wave□function□with□classical□and□quantum□ □
□□□□□□□□□□□□□□□□□ potential□for□g=1/100 □□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□□ 
 Classical potential 
 Quantum potential 
 ψ (x5) 
X
Figure 1. Inverse square potential. Classical potential, quan-
tum potential and groundstate wave function.
7
The celebrated semi-classical WKB formula [Wentzel, Kramers and Br il-
louin (1926)] gives approximate solutions to wave function and tunn eling am-
plitude. As an example let us consider in 1-D a system with parity symme tric
”conﬁnement” potential (potential goes to + ∞when|x| → ∞). Assume that
the system is in the ground state. The WKB formula gives an approxim ate
expression for wave function at energy E=Egr
ψWKB(x) =A
(2m[V(x)−Egr])1/4exp/bracketleftbigg
−1
/planckover2pi1/integraldisplayx
x0dx′/radicalBig
2m[V(x′)−Egr]/bracketrightbigg
.
Comparingwiththeresultsfromsect.4, weseethattheWKBformula becomes
exact, when replacing parameters of classical action by those of t he quantum
action
m→˜m , V(x)→˜V(x),
and replacing the rational term in front by a constant (wave funct ion normal-
isation).
5.Excited states of hydrogen atom. Can the quantum action also give
analytical results for excited states? The answer is yes, if we cons ider excited
states being lowest energy states of a conserved quantum numbe r. As an
example let us consider the radial motion of the hydrogen atom in a se ctor
of ﬁxed angular momentum l >0. The potential has a centrifugal plus a
Coulomb term. Let us consider angular motion to be quantized. We ke ep the
angular momentum quantum number lﬁxed. In the Feynman-Kac limit the
transition amplitude is projected onto the state of lowest energy c ompatible
with quantum number l. The states of the hydrogen atom arecharacterized by
the quantum numbers n(principal quantum number) l(angular momentum)
wheren=l+1. There is also the magnetic quantum number m. The radial
potential is given by
Vl(r) =/planckover2pi12l(l+1)
2mr2−e2
r.
The exact energy of the excited states is given by
El=−EI
n2=−EI
(l+1)2, EI=me4
2/planckover2pi12≈13.6eV(ionisation energy) .
The corresponding wave function is given by
φl(r) =1
Nl/parenleftbiggr
a0/parenrightbiggl
exp/bracketleftbigg
−r
(l+1)a0/bracketrightbigg
, a0=/planckover2pi12
me2(Bohr radius) .
For the quantum action we make an ansatz
˜m=m ,˜Vl(r) =µ/r2−ν/r .
8 H. KR¨OGER
A transformation law [similar to Eq.(6)] determines the parameters o f the
q-action,
µ=/planckover2pi12
2ml2, ν=e2l
l+1,
and the minimum of the quantum potential gives exactly the excitatio n ener-
gies
El=˜Vmin
l=−me4
2/planckover2pi121
(l+1)2=−EI
(l+1)2.
Again the wave function can be expressed in terms of the quantum a ction in a
way similar to Eq.(5), and reproduces the exact wave function φl. Moreover,
we observe that the excited state wave function φl(r) has its maximum where
the quantum potential ˜Vlhas its minimum. The quantum action has the same
structure as the classical action. Both, the centrifugal and the Coulomb term
get tuned.
6.Quantum instantons. Weconsiderquantummechanicsin1-D.Aparticle
of massmmoves in a quartic potential of double well shape given by
V(x) =1
2−x2+1
2x4,
andm= 1,/planckover2pi1= 1. The potential minima lie at a=±1. The potential barrier
has the height B= 1/2. The double well potential has a classical instanton
solution. It is obtained by solving the Euler-Lagrange equations of m otion of
the Euclidean classical action, with the initial conditions x(t=−∞) =−a,
˙x(t=−∞) = 0,
xcl
inst(t) =atanh[/radicalbig
2/m A a t ],
whereA= 1/√
2. The classical instanton goes from x(t=−∞) =−ato
x(t= +∞) = +a(see Fig.[2]).
For the quantum action we make the following ansatz,
˜S=/integraldisplay
dt1
2˜m˙x2−4/summationdisplay
k=0˜vkxk.
For example, at (imaginary) transition time T= 0.5 we ﬁnd the following
parameters for the quantum action ˜ m= 0.9961(2), ˜v0= 1.5710(17), ˜v1=
0.000(2), ˜v2=−0.745(6), ˜v3= 0.000(2), ˜v4= 0.493(3). By adding a constant,
the quantum potential can be expressed as a positive double well wit h minima
located at ±˜awith ˜a= 0.869(6). It has a barrier height ˜B= 0.281. It displays
”degenerate vacua”. Compared with the classical potential it is so fter, i.e. its
potential minima are closer to the origin and its barrier is also lower. Th e
quantum potential also has an instanton solution, corresponding t oT= 0.5,
given by
xT=0.5
inst(t) = ˜atanh[/radicalbig
2/˜m˜A˜a t]≈0.869 tanh[0.865t].
9
Similarly, we ﬁnd an instanton solution for any larger value of T. The quan-
tum instanton is obtained in the asymptotic limit T→ ∞. The evolution of
the instantons under variation of T, i.e. under variation of the temperature,
is depicted in Fig.[2]. It shows the transition from the classical instant on (at
inﬁnite temperature) to the quantum instanton (at zero tempera ture). One
observes that the quantum instanton is much softer than the clas sical instan-
ton.
−10 −5 0 5 10
t −1−0.500.51x(t) Quantum Instanton 
Double well potential 
T=0.0 (Classical Instanton)  
T=0.2 M.C.
T=0.4 M.C. 
T=0.6 M.C.
T=0.8 M.C.
T=1.0 S.E.
T=2.0 E.S.
T=3.0 E.S.
T=4.0 E.S.
T=5.0 E.S.
T=6.0 E.S.
T=7.0 E.S.
T=8.0 E.S. 
T=9.0 E.S.
Figure 2. Instantons in double well potential.
7.Quantum chaos. The hydrogen atom in the presence of strong magnetic
ﬁelds has been explored experimentally and theoretically [17, 19, 20]. It dis-
plays quantum chaos via disorder in the spectrum. In the regime of h igh lying
excited states, where the system becomes semi-classical, Gutzwille r’s trace for-
mula has been applied successfully by Wintgen [20] to extract periodic orbit
information. Here we suggest to explore quantum chaos via the pha se space
generated by the quantum action. As a prototype system, we con sider a 2-dim
non-integrable Hamiltonian, with a classically chaotic counter part.
Our deﬁnition of quantum chaos is based on the concept of ’some’ ph ase
space related to quantum mechanics. The novel idea is to introduce this phase
space via the quantum action. Second, the quantum action depend s on the
transition time T, however, for large Tit converges asymptotically. More-
over, in this regime the existence of the quantum action has been es tablished
10 H. KR¨OGER
rigorously. Theregimeoflarge Tmakes physical sense, becausetheproperdef-
inition of Lyapunov exponents, one of the characteristics of chao tic dynamics,
involves the large time limit.
As an example, let us consider the following Hamiltonian
S=/integraldisplayT
0dt1
2m(˙x2+ ˙y2)+V(x,y)
V(x,y) =v2(x2+y2)+v22x2y2
m= 1, v2= 0.5, v22= 0.05.
The parameter v22controls the deviation from integrability. At v22= 0 the
system is integrable. The system corresponds to two uncoupled os cillators, the
corresponding quantum action then coincides with the classical act ion. One
expects that tuning the parameter v22, while keeping the other parameters
ﬁxed, thesystemwillbecomemorechaoticwhenincreasing v22. Itisinteresting
to ask: Does the quantum system also become more chaotic?
For the quantum action, we have made the following ansatz: It reﬂe cts the
time-reversal symmetry, parity conservation and symmetry und er exchange
x↔y:
˜S=/integraldisplayT
0dt1
2˜m(˙x2+ ˙y2)+˜V(x,y),
˜V= ˜v0+ ˜v2(x2+y2)+ ˜v22x2y2+ ˜v4(x4+y4).
We also have included in the quantum action terms like ˙ x˙y,xy,xy3+x3y,
x2y4+x4y2,x4y4. Numerically, those coeﬃcients were found to be small
(compared to machine precision or zero within error bars).
An example of Poincar´ e sections for the classical system and the q uantum
systen are shown in Fig.[3]. The quantum Poincar´ e section correspo nds to
transition time T= 4.5. The energy has been chosen to be E= 10. One
can observe some diﬀerences between the classical and the quant um system.
Further numerical studies [10] have shown the following behavior: For small
v22, Poincar´ e sections of classical and quantum physics are quite simila r. With
increase of energy, both display mixed dynamics and become more ch aotic.
Also, with increase of energy diﬀerences between classical and qua ntum phasse
spacebecome morepronounced. Islands ofregular behavior diﬀer inshapeand
position. Quantitatively, one observes that the value of ˜ v22(quantum action)
is smaller than the corresponding value v22of the classsical action. Because
this parametyers measures the deviation from integrability, this hin ts to the
possibility that the quantum system is ”less” chaotic thanthe classic al system.
A more detailed analysis is called for.
8.Outlook: Further use of quantum action. In cosmology, the inﬂation-
ary scenario describes the early universe. Inﬂation involves poten tials with
11
Figure 3. 2-D anharmonic oscillator. Poincar´ e section of clas-
sical action vs. quantum action.
several minima and instanton solutions. The instanton starts out a s a quan-
tum instanton and eventually turns into a classical instanton. This h as eﬀects
on the subsequent formation of galaxies. The quantum action allows to search
for new minima inthe quantum potential, which may beabsent in the clas sical
potential.
In condensed matter physics, superconducting quantum interfe rence devices
(SQUID)havebeenusedbyFriedmanetal. todemonstrateexperim entallythe
phenomenon of quantum superposition in macroscopic states. This involves
12 H. KR¨OGER
Josephson junctions. The SQUID potential has a double-well stru cture. The
quantum action allows to construct a quantum potential and analyz e quantum
superposition in terms of such quantum potential.
In atomic physics, Steck et al. and Hensinger et al. have demonstra ted
experimentally the phenomenon of dynamical tunneling (where the c lassical
transition is forbidden due to some conserved quantity diﬀerent fr om energy).
It has been realized by arrays of cold atoms. It has been observed that the
presence of quantum chaos enhances the dynamical tunneling tra nsition. It
would be interesting to reexamine dynamical tunneling using the phas e space
portrait constructed from the quantum action.
In chemistry, the process of chemical binding of macromolecules of ten in-
volves potentials of a double well structure. The quantum action ca n be used
to study the formation of such macromolecules.
"""

In [6]:
dezen_flores_18 = " ".join([
    introduction,
    methods
])

In [7]:
len(nltk.word_tokenize(dezen_flores_18))

2350

In [18]:
def generate_summary(input_text: str, device: str='cpu') -> str:
    """
    Generate summary from Arxiv research article.

    """
    # Start clock
    start_time = time.time()

    # Set model and torch method
    model_name = [
                "sambydlo/bart-large-scientific-lay-summarisation",
                "com3dian/Bart-large-paper2slides-summarizer",
                "philschmid/bart-large-cnn-samsum"
            ]
    device = torch.device(device)
    tokenizer = BartTokenizer.from_pretrained(model_name[0])
    model = BartForConditionalGeneration.from_pretrained(model_name[0]).to(device)

    try:
        # Check length of article and batch if necessary
        batches = batch_input_text(input_text)
        # Summarize batches
        sub_summaries = []
        for i in range(len(batches)):
            output = model.generate(input_ids=tokenizer.encode(batches[i], return_tensors="pt").to(device),
                                    max_length=500, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
            summary = tokenizer.decode(output[0])
            sub_summaries.append(summary)
        # print(sub_summaries)
        
        # Combine child summaries & remove specified separators
        joined_summaries = " ".join(sub_summaries)
        # print(joined_summaries)
        parent_input = joined_summaries.replace("</s><s><s><s>", "").replace("</s>", "")
        print(parent_input)
        # Obtain final summary
        output = model.generate(input_ids=tokenizer.encode(parent_input, return_tensors="pt").to(device),
                                max_length=500)
        summary = tokenizer.decode(output[0])
        summary = summary.replace("</s><s><s><s>", "").replace("</s>", "")
    except:
        pass

    # Elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")

    return summary

def generate_summary_v2(input_text: str, device: str='cpu') -> str:
    """
    Generate summary from Arxiv research article.

    """
    # Start clock
    start_time = time.time()

    # Set model and torch method
    model_name = [
                "sambydlo/bart-large-scientific-lay-summarisation",
                "com3dian/Bart-large-paper2slides-summarizer",
                "philschmid/bart-large-cnn-samsum"
            ]
    device = torch.device(device)
    tokenizer = BartTokenizer.from_pretrained(model_name[0])
    model = BartForConditionalGeneration.from_pretrained(model_name[0]).to(device)

    try:
        # Check length of article and batch if necessary
        batches = batch_input_text(input_text)
        # Summarize batches
        sub_summaries = []
        for i in range(len(batches)):
            output = model.generate(input_ids=tokenizer.encode(batches[i], return_tensors="pt").to(device),
                                    max_length=500, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
            summary = tokenizer.decode(output[0])
            sub_summaries.append(summary)
        # print(sub_summaries)
        
        # Combine child summaries & remove specified separators
        joined_summaries = " ".join(sub_summaries).replace("</s><s><s><s>", "").replace("</s>", "")

    except:
        pass

    # Elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")

    return joined_summaries

def batch_input_text(input_text: str, batch_size: int=500) -> list:
    """
    :param input_text: str, research paper in full
    :param batch_size: int, 80% of max input tokens 
    :return: list, batched input text
    """
    try:
        tokens = nltk.word_tokenize(input_text)
        n_batches = math.ceil(len(tokens) / batch_size)

        if len(tokens) > batch_size:
            batches = [" ".join(tokens[(i * batch_size):((i + 1) * batch_size)]) 
                    for i in range(n_batches)]
        else:
            batches = [" ".join(tokens)]
    except Exception as e:
        raise e

    return batches

def resize_summaries_for_bart_input() -> str:
    """ 
    Resize the sub-batches if the sum of the batches 
    is greater than BART input maximum: 1024 tokens:
    :return: 
    """
    # try:
        
    

In [19]:
summary = generate_summary_v2(test_doc)
print(f"Output token length: {len(nltk.word_tokenize(summary))}")
print(summary)

Elapsed time: 158.18351697921753 seconds
Output token length: 915
Quantum physics is a branch of classical physics, which plays an important role in macroscopic physics, chemistry and biology. However, there are phenomena occuring in the quantum world which do not have a counter part in the classical world. Examples are entanglement, according to Schr¨ odinger themost peculiar property occuring inquantum mech anics, orthe condensationofvery coldatomspredictedbyEinsteinandBose ( Bo se-Einstein condensate ). Another example is nonlinear dynamics and chaos, da ting back to the work of Poincar´ e and its modern descendents - quantum chaos Chaos is a well-known phenomenon in classical physics. Its origin in terms of non linear dynamics has been investigated thoroughly in theoretical physics a nd math- ematics. If one considers nature at the level of atoms, where the laws of quantum mechanics ( Q.M. ) hold, chaotic phenomena are by far less we ll un- derstood than in classical ( macroscopic 

In [22]:
output = model.generate(input_ids=tokenizer.encode(summary, truncation=True, return_tensors="pt").to(device),
                        max_length=1024, length_penalty=2.0, num_beams=4, early_stopping=True)
summary_final = tokenizer.decode(output[0])
print(summary_final)

</s><s>Quantum physics is a branch of physics which plays an important role in macroscopic physics, chemistry and biology. However, there are phenomena occuring in the quantum world which do not have a counter part in the classical world. Examples are entanglement, according to Schr¨ odinger themost peculiar property occuring inquantum mech anics, orthe condensationofvery coldatomspredictedbyEinsteinandBose ( Bo se-Einstein condensate ). Another example is nonlinear dynamics and chaos, da ting back to the work of Poincar´ e and its modern descendents -</s>


In [23]:
len(nltk.word_tokenize(summary_final))

98

In [35]:
%%time
batches = batch_input_text(introduction, batch_size=200)
len(introduction)
# s = 0
# for i in batches:
#     s += 1
#     print(f"Batch {s} token count: {len(i)}")
#     print(i)

4
CPU times: total: 0 ns
Wall time: 5.99 ms


4407

In [ ]:
output = model.generate(input_ids=tokenizer.encode(introduction, return_tensors='pt').to(device), max_length=350)
tokenizer.decode(output[0])

'</s><s>Galactic black hole X-ray binaries (BHBs ) show several states of outburst distinguished by luminosity, spectral shape and variability (see for example, McClintock & Remillard (2006) and Done, Gierlinski, & Kubota (2007)). In particular, at their highest luminosities the spectra contains a steep power law component with photon index Γ > 2.4 (McClintock and Remillard2006). These energetically significant power law tails begin at the spectral peak (≈ 10 keV) and could extend into the MeV regime (Ling & Wheaton 2005; Grove et al. 1998). Moreover, this steep power</s>'

## sambydlo/bart-large-scientific-lay-summarisation
### Finetuning

In [4]:
# Test as pretrained model 
model_name = [
                "sambydlo/bart-large-scientific-lay-summarisation",
                "haining/scientific_abstract_simplification",
                "philschmid/bart-large-cnn-samsum"
            ]

tokenizer = BartTokenizer.from_pretrained(model_name[0])
model = BartForConditionalGeneration.from_pretrained(model_name[0]).to(device)

In [9]:
sci_lay_sum = generate_summary(introduction)
print(sci_lay_sum)

TypeError: generate_summary() got an unexpected keyword argument 'batch_size'

In [ ]:
# Test w/ pipeline
summarizer = pipeline("summarization", model="sambydlo/bart-large-tomasg25/scientific_lay_summarisation")


In [ ]:
{
"cache_dir": "opt/ml/input",
"dataset_config_name": "plos",
"dataset_name": "tomasg25/scientific_lay_summarisation",
"do_eval": true,
"do_predict": true,
"do_train": true,
"fp16": true,
"learning_rate": 5e-05,
"model_name_or_path": "facebook/bart-large",
"num_train_epochs": 1,
"output_dir": "/opt/ml/model",
"per_device_eval_batch_size": 4,
"per_device_train_batch_size": 4,
"predict_with_generate": true,
"seed": 7
}

## ArtifactAI/phi-2-arxiv-physics-instruct

In [ ]:

from huggingface_hub import notebook_login
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

model = "ArtifactAI/phi-2-arxiv-physics-instruct"

model = AutoModelForCausalLM.from_pretrained(base_model, trust_remote_code= True)
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

def generate(prompt):
  inputs = tokenizer(f'''Below is an instruction that describes a task. Write a response that appropriately completes the request If you are adding additional white spaces, stop writing".\n\n### Instruction:\n{prompt}.\n\n### Response:\n ''', return_tensors="pt", return_attention_mask=False)
  streamer = TextStreamer(tokenizer, skip_prompt= True)
  _ = model.generate(**inputs, streamer=streamer, max_new_tokens=500)
  
generate("what is an atom?")


: 

## ArtifactAI/led_base_16384_arxiv_summarization